In [ ]:
from getpass import getpass
from coreapi.auth import BasicAuthentication
from quantuminspire.api import QuantumInspireAPI
from quantuminspire.credentials import load_account, get_token_authentication, get_basic_authentication
from quantuminspire.api import QuantumInspireAPI
# from quantuminspire.projectq.backend_qx import QIBackend

import numpy as np
import pandas as pd
import plotly.express as px

In [ ]:
print('Enter mail address')
email = input()

print('Enter password')
password = getpass()

server_url = r'https://api.quantum-inspire.com'
authentication = BasicAuthentication(email, password)
qi = QuantumInspireAPI(server_url, authentication)

In [ ]:
#https://stackoverflow.com/questions/9600801/evenly-distributing-n-points-on-a-sphere
#Create n evenly distributed points on a sphere

num_pts = 1000
indices = np.arange(0, num_pts, dtype=float) + 0.5

theta = np.arccos(1 - 2*indices/num_pts)
phi = (np.pi * (1 + 5**0.5) * indices)%(2*np.pi)
coords=[]

for theta_elem,phi_elem in zip(theta,phi):
    coords.append((theta_elem,phi_elem))

x, y, z = np.cos(phi) * np.sin(theta), np.sin(phi) * np.sin(theta), np.cos(theta);

# Plot the surface.
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, template="plotly_white")

fig.show()

In [ ]:
def parameterized_QACM(loc):
    θ,ϕ=loc
    qasm = '''
    version 1.0

    qubits 5

    # initialize the state
    prep_z q[2]
    Ry q[2], {0}
    Rz q[2], {1}
    

    #preparation
    Ry q[0], 1.107149
    #rewrite CNOT q[0],q[4] and CNOT q[4],q[0] usign nearest neighbors
    SWAP q[0],q[2] 
    CNOT q[2],q[4]

    Ry q[4], 0.729728
    CNOT q[4],q[2]
    SWAP q[0],q[2]

    Ry q[0], 0.463648

    #copying
    CNOT q[2], q[0]
    CNOT q[2], q[4]
    CNOT q[0], q[2]
    CNOT q[4], q[2]

    #Rotate back and measure
    Rz q[0], {2}
    Ry q[0], {3}
    Measure_z q[0]
    '''
    return qasm.format(θ,ϕ,-ϕ,-θ)

In [ ]:
npoints=100
N_shots=4096
backend_type = qi.get_backend_type_by_name('Starmon-5')
target_points=np.array(coords)
# save the data, optional
np.savetxt("target_points_starmon.csv",target_points)

In [ ]:
# iterate over the sphere, run task

copied_data=[]
index=0

for points in target_points:
    qasm=parameterized_QACM(points)
    result = qi.execute_qasm(qasm, backend_type=backend_type, number_of_shots=N_shots)
    hist=result.get('histogram', {})
    copied_data.append([hist["0"],hist["1"]])
    index=index+1
    print(str(index)+"/"+str(npoints))
    print("Execution time: ", result.get('execution_time_in_seconds',{}))
    print(points)
    print([hist["0"],hist["1"]])
    with open('results_starmon.txt', 'a') as file:
        file.write(str(points[0])+"\t"+str(points[1])+"\t"+str(hist["0"])+"\t"+str(hist["1"])+"\n")
    

copied_data=np.array(copied_data)

In [ ]:
data_sheet=pd.DataFrame(data=np.hstack((target_points,copied_data)),columns=["θ","ϕ","prob_0","prob_1"])

In [ ]:
data_sheet.head()

In [ ]:
# save to excel, optional
data_sheet.to_excel("data_sheet_starmon.xlsx")

In [ ]:
data_sheet=pd.read_excel("data_sheet_starmon.xlsx")

In [ ]:
# Make data.
thetas=data_sheet["θ"]
phis=data_sheet["ϕ"]
x, y, z = np.cos(phis) * np.sin(thetas), np.sin(phis) * np.sin(thetas), np.cos(thetas);

# Plot the surface. F_measured/F_theory
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, color=data_sheet.prob_0/(5/6), template="plotly_white")

fig.show()

